In [ ]:
import pandas as pd
import time
import datetime
import numpy as np


import prob_tree
import kTree
import csv
from fractions import Fraction
import graphviz
import tree
from kTree import ktree_main

import sys
import generators
import numpy as np
import math
from data import Data
from datetime import datetime, timedelta

import os
import pickle
import json



data_train1 = pd.read_csv('./input_data/Future/20190620/IH1906.CFFEX.txt',header=0)

In each trading day, the data is updated every 0.5s. However, if the market data doesn't change, the data is not updated. That's why the time interval could be more than 0.5s, e.g. 1.5s. So for each trading day, we need to filling the missing data using the previous data, so that the data time interval is consistent and equals to 0.5s. Then we need to calculate the fair prices and the corresponding returns of the main contract. To process the data efficiently, we save the the fair prices of the main contract in txt files.

*How to determine the main contract? For index futures, the third Friday of the month is the last trading day. Before the third Friday in this month, the main index future contract is exactly the future conract maturing in this month; after the third Friday (include), the main index future contract is the future conract maturing in the next month. E.g. on 14 June, 2019, the main futures contract of IH is IH1906; on 27 June, 2019, the main futures contract of IH is IH1907.

In [ ]:
###Read the data of the main futures conrtact in each trading day.

import os
date=20180101
num_train_day=0
date_unit0=0
change_day=31
date_list=[]
while date<20190628:
    path = '/Volumes/TOSHIBA EXT/其他/Luoshu/L2/'+str(date)
    date_date = datetime(year=int(str(date)[0:4]), month=int(str(date)[4:6]), day=int(str(date)[6:8]))
    if os.access(path, os.F_OK):
        num_train_day += 1
        date_list.append(date)
        date_year = int(str(date)[2:4])
        date_month = int(str(date)[4:6])
        date_day = int(str(date)[6:8])
        if ((date_date.weekday() == 4) & (14<=date_day) & (date_day<=21)): #judge if this day is the last trading day in the month
            change_day = date_day
            print(change_day)
        if date_day>=change_day: #when the current day is after the last trading day in the month
            if date_month==12:
                path=path+'/IH'+str(date_year+1)+str(1).zfill(2)+'.CFFEX.txt'
                locals()['data_train'+str(num_train_day)]= pd.read_csv(path,header=0)
                print(path)
            else:
                path=path+'/IH'+str(date_year)+str(date_month+1).zfill(2)+'.CFFEX.txt'
                locals()['data_train'+str(num_train_day)]= pd.read_csv(path,header=0)
                print(path)
        else:  #when the current day is before the last trading day in the month
            path=path+'/IH'+str(date_year)+str(date_month).zfill(2)+'.CFFEX.txt'
            locals()['data_train'+str(num_train_day)]= pd.read_csv(path,header=0)
            print(path)
    
    date_date =  date_date+timedelta(days=1)  
    date = 10000*date_date.year + 100*date_date.month + date_date.day
    #print(date)
        
print(num_train_day)   

In [ ]:
###Extract the data in trading hours, i.e. those with non-zero ask & bid quantities
for i in range(1,num_train_day+1):
    name='data_train'+str(i)
    locals()['data_train_valid'+str(i)]=locals()[name].loc[locals()[name][locals()[name]['Ask1Qty']!=0].index,]
    locals()['data_train_valid'+str(i)]=locals()['data_train_valid'+str(i)].loc[locals()['data_train_valid'+str(i)][locals()['data_train_valid'+str(i)]['Bid1Qty']!=0].index,]
    locals()['data_train_valid'+str(i)]=locals()['data_train_valid'+str(i)].set_index('ExchangeTime')
    locals()['data_train_valid'+str(i)]['Ask1Price']=pd.to_numeric(locals()['data_train_valid'+str(i)]['Ask1Price'])
    locals()['data_train_valid'+str(i)]['Bid1Price']=pd.to_numeric(locals()['data_train_valid'+str(i)]['Bid1Price'])
    locals()['data_train_valid'+str(i)].drop(locals()['data_train_valid'+str(i)].index[0],inplace=True)
    print(i)
print(data_train_valid1.dtypes[1:18])
print(len(data_train_valid1))
data_train_valid1.head()

In [ ]:
###Fill in the missing data using previous data, 
for i in range(1,num_train_day+1):
#154 bad format 
# for i in range(155,num_train_day+1):
    name='data_train_valid'+str(i)
    price_train = (locals()[name]['Ask1Price']*locals()[name]['Bid1Qty']+locals()[name]['Bid1Price']*locals()[name]['Ask1Qty'])/(locals()[name]['Ask1Qty']+locals()[name]['Bid1Qty'])
    j=0
    while True:
        if j == (len(price_train)-1): break
        if ((price_train.index[j+1]-price_train.index[j]>500000) & (price_train.index[j+1]-price_train.index[j]<600000000)):
            diff_num = int((price_train.index[j+1]-price_train.index[j])/500000)
            for ins in range(1,diff_num):
                above = price_train[:j+ins]
                below = price_train[j+ins:]
                insert = price_train[j:j+1]
                above_ind = price_train.index.values[:j+ins].tolist()
                below_ind = price_train.index.values[j+ins:].tolist()
                insert_ind = price_train.index[j]+500000*ins
                price_train = above.append(insert,ignore_index=True).append(below,ignore_index=True)
                above_ind.append(insert_ind)
                above_ind.extend(below_ind)
                price_train.index=above_ind
        j=j+1
    freq = 1
    price_index = [x * freq for x in list(range(int(len(price_train)/freq)))]
    price_train_ = price_train.iloc[price_index]

    price_train_1 = price_train_[:(len(price_train_)-1)]
    price_train_2 = price_train_[1:len(price_train_)]
    price_train_1.index = price_train_2.index
    return_train = (price_train_2/price_train_1-1)*100
    locals()['return_train'+str(i)] = return_train.tolist()
    
    name = '2mid_volume_train_IH_'+str(date_list[i-1])+'.txt'
    f = open(name, "wb")  
    pickle.dump(price_train, f)
    f.close()
    print(i)
len(return_train1) 

In [ ]:
#Plot daily price
freq=1
date_start=20190101
date_end=20190104
date=date_start
while date<date_end:
    path = './input_data/Future/mid_volume_train_IH_1min_'+str(date)+'.txt'
    date_date = datetime(year=int(str(date)[0:4]), month=int(str(date)[4:6]), day=int(str(date)[6:8]))
    if os.access(path, os.F_OK):
        #path = './input_data/Future/2mid_volume_train_IC_'+str(date_test)+'.txt'
        f = open(path, "rb")  
        price_test = pickle.load(f)
        price_index = [x * freq for x in list(range(int(len(price_test)/freq)))]
        price_test_ = price_test.iloc[price_index]
        price_test_list=price_test_.tolist()
        import matplotlib.pyplot as plt
        plt.plot(price_test_list)
        plt.xlabel('t')
        plt.ylabel('pnl')
        name="price_IH_"+str(date)
        plt.title(name)
        #plt.xticks(test_time)
        plt.savefig(name+".png")
        plt.figure()
    date_date =  date_date+timedelta(days=1)  
    date = 10000*date_date.year + 100*date_date.month + date_date.day

In [ ]:
###Plot price of previous n days
import matplotlib.pyplot as plt

freq=120
num_days=120
date_start=20190327
date_end=20190328
date=date_start
while date<date_end:
    path = './input_data/Future/mid_volume_train_IH_1min_'+str(date)+'.txt'
    date_date = datetime(year=int(str(date)[0:4]), month=int(str(date)[4:6]), day=int(str(date)[6:8]))
    if os.access(path, os.F_OK):
        #path = './input_data/Future/2mid_volume_train_IC_'+str(date_test)+'.txt'
        date_train=date
        i=0
        price_test_list=[]
        while i<num_days:
            date_date_train = datetime(year=int(str(date_train)[0:4]), month=int(str(date_train)[4:6]), day=int(str(date_train)[6:8]))
            date_date_train =  date_date_train-timedelta(days=1) 
            date_train = 10000*date_date_train.year + 100*date_date_train.month + date_date_train.day
            path = './input_data/Future/mid_volume_train_IH_1min_'+str(date_train)+'.txt'
            if os.access(path, os.F_OK):
                print(date_train)
                f = open(path, "rb")  
                price_test = pickle.load(f)
                price_index = [x * freq for x in list(range(int(len(price_test)/freq)))]
                price_test_ = price_test.iloc[price_index]
                locals()['price_test_'+str(i)]=price_test_.tolist()
                i+=1
        while i>0:
            price_test_list.extend(locals()['price_test_'+str(i-1)]) 
            i-=1
        plt.plot(price_test_list)
        plt.xlabel('t')
        plt.ylabel('pnl')
        name="price_previous"+str(num_days)+"days_IH_"+str(date)
        plt.title(name)
        #plt.xticks(test_time)
        plt.savefig(name+".png")
        plt.figure()
    date_date =  date_date+timedelta(days=1)  
    date = 10000*date_date.year + 100*date_date.month + date_date.day